In [47]:
from functools import reduce
import func_helper.func_helper.iterator as it
import func_helper.func_helper.transducer as ts

from func_helper import compose, pip, memoize, trace, Y, recursiveExtender, identity

import numpy as np

## Adaptor

In [45]:
# flip

def flip(func):
    """
    assert(flip(identity)(1) == [1])
    assert(flip(identity)([1]) == [1])
    assert(flip(identity)(1,2,3) == [2,3,1])
    assert(flip(identity)([1,2,3]) == [2,3,1])
    """
    def f(*arg):
        first, *rest = arg[0] if type(arg[0]) in (list, tuple) else arg
        return func([*rest, first])
    return f


assert(flip(identity)(1) == [1])
assert(flip(identity)([1]) == [1])
assert(flip(identity)(1,2,3) == [2,3,1])
assert(flip(identity)([1,2,3]) == [2,3,1])

[2, 3, 1]


In [46]:
# over

def over(*funcs):
    """
    extent = over(np.min, np.max)
    assert(list(extent([0,1,2,3,4,5])) == [0,5])
    """
    return lambda *arg: map(lambda f: f(*arg), funcs)

extent = over(np.min, np.max)

assert(list(extent([0,1,2,3,4,5])) ==[0,5])

## Function

In [2]:
add = lambda x: lambda y: x+y
multiple = lambda x: lambda y: x * y
square = lambda x: x**2

equivarent = lambda x: square(multiple(3)(add(5)(x)))

composed = compose(square, multiple(3), add(5))

assert(equivarent(0) == composed(0))
assert(equivarent(-1) == composed(-1))
assert(equivarent(10) == composed(10))

In [3]:
(lambda x: x*2)(2)

4

In [3]:
traced_add = trace(lambda x,y: x+y)
traced_add(2,3)

called with argument (2, 3)


5

In [4]:
traced_square = trace(lambda a: a**2)
traced_square(5)

called with argument (5,)


25

## recursive function

In [5]:


def fib_template(f):
    """
    F -> (a -> F.a)
    """
    def func(arg):
        n = int(arg)
        if n < 2:
            return 1
        else:
            return f(n-1) + f(n-2)
    return func
            
fib = Y(fib_template)
traced_fib=Y(recursiveExtender(trace)(fib_template))

In [6]:
fib(5)

8

In [21]:
traced_fib(5)

called with argument 4
called with argument 3
called with argument 2
called with argument 1
called with argument 0
called with argument 1
called with argument 2
called with argument 1
called with argument 0
called with argument 3
called with argument 2
called with argument 1
called with argument 0
called with argument 1


8

In [7]:
memo_fib = Y(recursiveExtender(memoize())(
    recursiveExtender(trace)(fib_template)
))
memo_fib(5)

called with argument (4,)
called with argument (3,)
called with argument (2,)
called with argument (1,)
called with argument (0,)
called with argument (1,)
called with argument (2,)
called with argument (3,)


8

In [8]:
memo_fib(50)

called with argument (49,)
called with argument (48,)
called with argument (47,)
called with argument (46,)
called with argument (45,)
called with argument (44,)
called with argument (43,)
called with argument (42,)
called with argument (41,)
called with argument (40,)
called with argument (39,)
called with argument (38,)
called with argument (37,)
called with argument (36,)
called with argument (35,)
called with argument (34,)
called with argument (33,)
called with argument (32,)
called with argument (31,)
called with argument (30,)
called with argument (29,)
called with argument (28,)
called with argument (27,)
called with argument (26,)
called with argument (25,)
called with argument (24,)
called with argument (23,)
called with argument (22,)
called with argument (21,)
called with argument (20,)
called with argument (19,)
called with argument (18,)
called with argument (17,)
called with argument (16,)
called with argument (15,)
called with argument (14,)
called with argument (13,)
c

20365011074

## transducer

In [2]:
ts.intoArray?

Signature: ts.intoArray(*funcs)
Docstring: <no docstring>
File:      c:\users\fmori\desktop\python3\func_helper\func_helper\transducer\transducer.py
Type:      function


In [6]:
ts.intoArray(
    ts.mapping(lambda a: a+ 2),
    ts.mapping(lambda a: a**2),
    ts.filtering(lambda a: a%2 == 0)
)(range(7))

[4, 16, 36, 64]

## iterator

In [2]:
it.mapping?

Signature: it.mapping(mapFunc)
Docstring: <no docstring>
File:      c:\users\fmori\desktop\python3\func_helper\func_helper\iterator\iterator.py
Type:      function


In [4]:
pip(
    it.mapping(lambda a: a+2),
    it.mapping(lambda a: a**2),
    it.filtering(lambda a: a%2 == 0),
    list
)(range(7))

[4, 16, 36, 64]

In [70]:
def is_all(pred):
    """
    pred: a -> bool
    arr: list, tuple
    
    assert(is_all(lambda x: x > 0)([1,2,3,4, 5]) is True)
    assert(is_all(lambda x: x > 0)((1,2,3,4, 0)) is not True)
    """
    return lambda arr: all(map(pred, arr))

assert(is_all(lambda x: x > 0)([1,2,3,4, 5]) is True)
assert(is_all(lambda x: x > 0)((1,2,3,4, 0)) is not True)

def is_any(pred):
    """
    pred:a -> bool
    arr: list, tuple
    
    assert(is_any(lambda x: x%2 is 0)([1,3,5,7,8]) is True)
    assert(is_any(lambda x: x%2 is 0)((1,3,5,7,9)) is not True)
    """
    return lambda arr: any(map(pred, arr))

assert(is_any(lambda x: x%2 is 0)([1,3,5,7,8]) is True)
assert(is_any(lambda x: x%2 is 0)((1,3,5,7,9)) is not True)


def all_equal(arr):
    """
    assert(all_equal([1,1,1]) is True)
    assert(all_equal((1,1,2)) is not True)
    """
    
    first, *rest = arr
    return is_all(lambda e: e == first)(rest)

assert(all_equal([1,1,1]) is True)
assert(all_equal((1,1,2)) is not True)
